In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

%matplotlib inline

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist_data = input_data.read_data_sets('/tmp/data/', one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
n_classes = 10
feature_count = 28 * 28

In [5]:
n_units_hl = (500, 500, 500)
batch_size = 300
precision = 32

In [6]:
type_str = 'float{0}'.format(precision)
with tf.name_scope('input'):
    x = tf.placeholder(type_str, shape=(None, feature_count), name='x')
    y = tf.placeholder(type_str, name='label')

In [7]:
print(x)
print(y)

Tensor("input/x:0", shape=(?, 784), dtype=float32)
Tensor("input/label:0", dtype=float32)


In [8]:
k_key_weights = 'weights'
k_key_biases = 'biases'
k_key_units = 'units'

In [9]:
def nn_model(data):
    layers = []
    real_n_units = list(n_units_hl)
    real_n_units.append(n_classes)
    for idx, n_units in enumerate(real_n_units):
        n = n_units
        m = feature_count
        if idx is not 0:
            m = n_units_hl[idx - 1]
        with tf.device('/gpu:0'):
            with tf.name_scope('layer'):
                layer = {k_key_weights: tf.Variable(tf.random_normal([m, n]), name='weight'),
                         k_key_biases: tf.Variable(tf.random_normal([n]), name='bias')}
                layers.append(layer)
    with tf.device('/gpu:0'):
        with tf.name_scope('layer'):
            for idx, layer in enumerate(layers):
                if idx is 0:
                    last_layer = data
                else:
                    last_layer = layers[idx - 1][k_key_units]
                name_kwargs = {'name':'output'} if idx is len(layers) - 1 else {'name':'neuron'}
                layer[k_key_units] = tf.add(tf.matmul(last_layer, layer[k_key_weights]), layer[k_key_biases], **name_kwargs)

                if idx is not len(layers) - 1:
                    layer[k_key_units] = tf.nn.relu(layer[k_key_units], name='activation_func')
    
    return layers[-1][k_key_units]

In [10]:
def train_nn(data):
    prediction = nn_model(data)
    with tf.name_scope('train'):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y), name='cost')
        optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    with tf.name_scope('performance'):
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'), name='acccuracy')

    hm_epochs = 10  # Number of iterations
    
    with tf.Session() as sess:
        start = time.time()
        sess.run(tf.global_variables_initializer())
        print('Initialized global variables...')
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist_data.train.num_examples/batch_size)):
                x_epoch, y_epoch = mnist_data.train.next_batch(batch_size)
                _, c_epoch = sess.run([optimizer, cost], feed_dict = {x: x_epoch, y: y_epoch})
                epoch_loss += c_epoch
            print('Epoch {0} completed out of {1} epochs, epoch_loss: {2}.'.format(epoch, hm_epochs, epoch_loss))
        print()
        
        print('Accuracy: {0:.2f}%'.format(accuracy.eval({x: mnist_data.test.images, y: mnist_data.test.labels}) * 100.0))
        end = time.time()
        print('Duration: {0:.2f}s'.format(end - start))
        
        # TensorBoard
        
        tb_writer = tf.summary.FileWriter('tensorboard/dnn')
        tb_writer.add_graph(sess.graph)

In [11]:
train_nn(x)

Initialized global variables...
Epoch 0 completed out of 10 epochs, epoch_loss: 873478.1030273438.
Epoch 1 completed out of 10 epochs, epoch_loss: 209211.9384765625.
Epoch 2 completed out of 10 epochs, epoch_loss: 126834.90715026855.
Epoch 3 completed out of 10 epochs, epoch_loss: 86511.73857116699.
Epoch 4 completed out of 10 epochs, epoch_loss: 60179.20980834961.
Epoch 5 completed out of 10 epochs, epoch_loss: 42832.63708496094.
Epoch 6 completed out of 10 epochs, epoch_loss: 30222.991861343384.
Epoch 7 completed out of 10 epochs, epoch_loss: 22147.379642486572.
Epoch 8 completed out of 10 epochs, epoch_loss: 14946.484138607979.
Epoch 9 completed out of 10 epochs, epoch_loss: 10084.081165075302.

Accuracy: 93.85%
Duration: 4.23s
